In [1]:
!apt-get update -y
!apt-get install -y tabix bcftools samtools
!pip install pyfaidx

import gzip
import numpy as np
import pandas as pd
import os
from pyfaidx import Fasta


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [2]:
# Skipped the 'subset to loci with matched transcript_ids in GRCh38.p14 GTF file' part from original paper

CLINVAR_VCF_URL="https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/archive_2.0/2024/clinvar_20240301.vcf.gz"
#GTF_URL="https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/latest_release/gencode.v49.annotation.gtf.gz"

DATA_DIR="/content/data"
!mkdir -p $DATA_DIR

!wget -O $DATA_DIR/clinvar.vcf.gz "$CLINVAR_VCF_URL"
#!wget -O $DATA_DIR/grch38_gtf.gtf.gz "$GTF_URL"

!ls -lh $DATA_DIR

!gunzip -c $DATA_DIR/clinvar.vcf.gz > $DATA_DIR/clinvar.vcf
#!gunzip -c $DATA_DIR/grch38_gtf.gtf.gz > $DATA_DIR/grch38_gtf.gtf

--2025-11-10 16:48:18--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/archive_2.0/2024/clinvar_20240301.vcf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100783681 (96M) [application/x-gzip]
Saving to: ‘/content/data/clinvar.vcf.gz’

/content/data/clinv 100%[===================>]  96.11M  77.9MB/s    in 1.2s    

2025-11-10 16:48:20 (77.9 MB/s) - ‘/content/data/clinvar.vcf.gz’ saved [100783681/100783681]

total 1.2G
-rw-r--r-- 1 root root 1.1G Nov 10 16:47 clinvar.vcf
-rw-r--r-- 1 root root  97M Mar  1  2024 clinvar.vcf.gz


In [3]:
!bcftools view $DATA_DIR/clinvar.vcf | head -n 60

[W::vcf_parse] Contig '1' is not defined in the header. (Quick workaround: index the file with tabix.)
##fileformat=VCFv4.1
##FILTER=<ID=PASS,Description="All filters passed">
##fileDate=2024-03-01
##source=ClinVar
##reference=GRCh38
##ID=<Description="ClinVar Variation ID">
##INFO=<ID=AF_ESP,Number=1,Type=Float,Description="allele frequencies from GO-ESP">
##INFO=<ID=AF_EXAC,Number=1,Type=Float,Description="allele frequencies from ExAC">
##INFO=<ID=AF_TGP,Number=1,Type=Float,Description="allele frequencies from TGP">
##INFO=<ID=ALLELEID,Number=1,Type=Integer,Description="the ClinVar Allele ID">
##INFO=<ID=CLNDN,Number=.,Type=String,Description="ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
##INFO=<ID=CLNDNINCL,Number=.,Type=String,Description="For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
##INFO=<ID=CLNDISDB,Number=.,Type=String,Description="Tag-value pairs of dis

In [4]:
!bcftools query -f '%CHROM\t%POS\t%ID\t%REF\t%ALT\t%INFO/ALLELEID\t%INFO/GENEINFO\t%INFO/CLNSIG\t%INFO/CLNREVSTAT\t%INFO/RS\t%INFO/CLNHGVS\n' \
  $DATA_DIR/clinvar.vcf > clinvar_raw.tsv 2>/dev/null
!head -n 10 clinvar_raw.tsv

1	69134	2205837	A	G	2193183	OR4F5:79501	Likely_benign	criteria_provided,_single_submitter	.	NC_000001.11:g.69134A>G
1	69581	2252161	C	G	2238986	OR4F5:79501	Uncertain_significance	criteria_provided,_single_submitter	.	NC_000001.11:g.69581C>G
1	69682	2396347	G	A	2386655	OR4F5:79501	Uncertain_significance	criteria_provided,_single_submitter	.	NC_000001.11:g.69682G>A
1	69769	2288999	T	C	2278803	OR4F5:79501	Uncertain_significance	criteria_provided,_single_submitter	.	NC_000001.11:g.69769T>C
1	69995	2351346	G	C	2333177	OR4F5:79501	Uncertain_significance	criteria_provided,_single_submitter	.	NC_000001.11:g.69995G>C
1	925946	1924157	C	G	1983057	SAMD11:148398	Uncertain_significance	criteria_provided,_single_submitter	.	NC_000001.11:g.925946C>G
1	925952	1019397	G	A	1003021	SAMD11:148398	Uncertain_significance	criteria_provided,_single_submitter	1640863258	NC_000001.11:g.925952G>A
1	925956	1543320	C	T	1632777	SAMD11:148398	Likely_benign	criteria_provided,_single_submitter	1342334044	NC_000001.11:

In [5]:
import pandas as pd
import re

cols = ["CHROM","POS","ID","REF","ALT","ALLELEID","GENEINFO","CLNSIG","CLNREVSTAT","RS","CLNHGVS"]
df = pd.read_csv("clinvar_raw.tsv", sep="\t", names=cols, comment="#", dtype=str)

# Filtering

# Keep nuclear chromosomes only
chr = df['CHROM'].str.match(r'^(chr)?([1-9]|1[0-9]|2[0-2]|X|Y)$')
df = df[chr].copy()
# Keep SNVs only
df = df[df['REF'].str.len() == 1]
df = df[df['ALT'].str.len() == 1].copy()
# Review status (at least two stars)
rev_re = re.compile(r'practice_guideline|reviewed_by_expert_panel|criteria_provided,_multiple_submitters')
df = df[df['CLNREVSTAT'].astype(str).str.contains(rev_re)]
# Exclude VUS and collapse CLNSIG
df = df[~df['CLNSIG'].astype(str).str.contains('Uncertain_significance', na=False)]
P = df['CLNSIG'].str.contains('Pathogenic|Likely_pathogenic', na=False)
B = df['CLNSIG'].str.contains('Benign|Likely_benign', na=False)
df = df[(P ^ B)]
df.loc[P, 'LABEL'] = 'Pathogenic/Likely_pathogenic'
df.loc[B, 'LABEL'] = 'Benign/Likely_benign'

df.to_csv("clinvar_filtered.tsv", sep="\t", index=False)
print("clinvar_filtered.tsv! rows:", len(df))

!head -n 10 clinvar_filtered.tsv

clinvar_filtered.tsv! rows: 192949
CHROM	POS	ID	REF	ALT	ALLELEID	GENEINFO	CLNSIG	CLNREVSTAT	RS	CLNHGVS	LABEL
1	935779	1167937	G	A	1153708	SAMD11:148398	Benign/Likely_benign	criteria_provided,_multiple_submitters,_no_conflicts	143282473	NC_000001.11:g.935779G>A	Benign/Likely_benign
1	944036	1154628	G	A	1131808	SAMD11:148398	Likely_benign	criteria_provided,_multiple_submitters,_no_conflicts	202045502	NC_000001.11:g.944036G>A	Benign/Likely_benign
1	963233	775485	C	T	696923	KLHL17:339451	Benign	criteria_provided,_multiple_submitters,_no_conflicts	61746776	NC_000001.11:g.963233C>T	Benign/Likely_benign
1	970514	710495	C	T	730039	PLEKHN1:84069	Benign/Likely_benign	criteria_provided,_multiple_submitters,_no_conflicts	41285810	NC_000001.11:g.970514C>T	Benign/Likely_benign
1	1013490	1185393	C	G	1175250	ISG15:9636	Benign	criteria_provided,_multiple_submitters,_no_conflicts	4615788	NC_000001.11:g.1013490C>G	Benign/Likely_benign
1	1013541	1185394	T	C	1175251	ISG15:9636	Benign	criteria_provided,_mul

In [6]:
# The original filtered ClinVar set is too large (192949 rows).
# Randomly sample from it to get a subset of a reasonable size
sample_df = df.sample(frac=0.05, random_state=42)
sample_df.to_csv("clinvar_subset.tsv", sep="\t", index=False)
# Export the dataset and later feed it into Evo 2

In [7]:
# Extract TP53 variants to get a TP53 test set
df_tp53 = df[df["GENEINFO"].astype(str).str.contains("TP53", case=True, na=False)].reset_index(drop=True)
print("Total TP53 variants:", len(df_tp53))
df_tp53.to_csv("clinvar_tp53_test.tsv", sep="\t", index=False)

Total TP53 variants: 610
